In [5]:
import os

In [6]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Kidney_Disease_MLFlow_DVC_Docker'

In [7]:
os.chdir("../")

In [8]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS'

In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pavi2803/Kidney_Disease_MLflow_DVC_Docker.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="pavi2803"
os.environ["MLFLOW_TRACKING_PASSWORD"]="295a833ef222c2a7970ad7a4b9ebfbf5b23ac5e8"

In [10]:
import tensorflow as tf

In [12]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS'

In [13]:
import os
os.chdir('/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Kidney_Disease_MLFlow_DVC_Docker')

In [14]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/pavi2803/Kidney_Disease_MLflow_DVC_Docker.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [21]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [26]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            # ✅ Save model locally to a temp folder
            local_model_path = "mlruns_temp_model"
            self.model.save(local_model_path)

            # ✅ Log the saved model folder as an artifact
            mlflow.log_artifacts(local_model_path, artifact_path="model")

            # ✅ Clean up
            shutil.rmtree(local_model_path)


In [28]:
# try:
#     config = ConfigurationManager()
#     eval_config = config.get_evaluation_config()
#     evaluation = Evaluation(eval_config)
#     evaluation.evaluation()
#     evaluation.log_into_mlflow()

# except Exception as e:
#    raise e

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    print(f"❌ Evaluation pipeline failed: {e}")
    raise


Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.9827 - loss: 0.0500
🏃 View run sincere-moose-217 at: https://dagshub.com/pavi2803/Kidney_Disease_MLflow_DVC_Docker.mlflow/#/experiments/0/runs/6b37613222ac43289fed86b3ada1433d
🧪 View experiment at: https://dagshub.com/pavi2803/Kidney_Disease_MLflow_DVC_Docker.mlflow/#/experiments/0
❌ Evaluation pipeline failed: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=mlruns_temp_model.


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=mlruns_temp_model.